In [27]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# set random seed for comparing the two result calculations
tf.set_random_seed(1)

# this is data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [28]:
# hyperparameters
lr = 0.001
training_iters = 1000
batch_size = 128

n_inputs = 28   # MNIST data input (img shape: 28*28)
n_steps = 28    # time steps
n_hidden_units = 128   # neurons in hidden layer
n_classes = 10      # MNIST classes (0-9 digits)

In [1]:
a = [1,2,3]
sum(a)/len(a)

2.0

In [30]:
graph = tf.Graph()
with graph.as_default():
    # input 
    x = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
    y = tf.placeholder(tf.float32, shape=[None, n_classes])
    
    #parameters
    weights = {
        'in':tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
        'out':tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
    }
    biases = {
        'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units,])),
        'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
    }
    
    #computations
    def RNN(X, weights, biases):
        #hidden layers to cell 
        #X ==> (128 batch * 28 steps, 28 inputs)
        X = tf.reshape(X, [-1, n_inputs])
        #X_in = (128 batch, 28 steps, 128 hidden)
        X_in = tf.reshape(tf.matmul(X, weights['in']) + biases['in'], shape=[-1, n_steps, n_hidden_units])
        
        #cell 
        cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
        init_state = cell.zero_state(batch_size, dtype=tf.float32)
        #init_state = tf.zeros([batch_size, cell.state_size])
        
        output, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state = init_state, time_major=False)
        
        #hidden layer for output as the final results
        outputs = tf.unstack(tf.transpose(output, [1,0,2]))
        results = tf.matmul(outputs[-1], weights['out']) + biases['out']
        
        return results
    
    #loss & Optimizer 
    pred = RNN(x, weights, biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = pred))
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)
    
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), tf.float32))

In [26]:
with tf.Session(graph=graph) as sess:
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([optimizer], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
            }))
        step += 1
    

0.164063
0.695313
0.773438
0.820313
0.765625
0.875
0.890625
0.890625
0.914063
0.859375
0.875
0.921875
0.929688
0.898438
0.921875
0.921875
0.945313
0.96875
0.945313
0.945313
0.96875
0.96875
0.953125
0.953125
0.945313
0.960938
0.945313
0.929688
0.945313
0.953125
0.945313
0.945313
0.953125
0.960938
0.953125
0.960938
0.960938
0.984375
0.976563
0.984375
